<목차>
- Image Segmetation을 위한 Foundation Model 모델 설계를 위한 Task 정의
- 데이터 수집 과정
  - Assisted-manual
  - Semi-automatic
  - Fully-automatic
- 모델 구조
  - Image Encoder
  - Prompt Encoder
  - Mask Decoder
  - 성능
- Everything 기능이란?
- 인상적인 후속 주제들
- Predictor

### Image Segmetation 기능을 하는 Foundation Model 모델 설계를 위한 Task 정의
이미지 Segmentation 모델(SAM)을 만들기 위한 Foundation Model을 설계하기 위해서 한 3가지 질문
1. What task will enable zero-shot generalization?  
2. What is the corresponding model architecture?
3. What data can power this task and model? 

- (처음 본 사물도 인식할 수 있는) zero-shot 일반화를 위한 Task는 어떤 Task인가?  
  --> `어떻하면 애매모호한 상황도 모두 대처할 수 있는 모델을 만들 수 있을까? Promptable Segmentation Task를 정의`.  
      점(point), 박스(box), 마스크(mask), 텍스트(text)를 입력으로 받을 수 있게 설계.  
  --> `모호한 Prompt가 주어졌을 때도 합리적인 mask를 출력해야 한다. 모호하다면 연관된 것을 다 확률로 나타낸다`.
  
- 이에 상응하는 모델 구조는 어떻게 되야 하는가?  
  --> Promt를 당연히 지원해야 한다.
  
- 어떤 데이터가 필요한가?  
  --> Promptable Segmentation Task를 위한 Segmentation Mask는 구하기 어렵다.

### 데이터 수집 과정
1. `Assisted-manual`
   - 공개된 Segmentation Dataset을 이용해 SAM 초기 학습
   - 웹 기반 인터페이스에서 초기 학습된 SAM을 이용해 데이터 생성
   - 새로 취득한 Data로 점진적 모델 학습(6회)
   - 120k 이미지로 부터 4.3M Mask 취득  
2. `Semi-automatic`
   - Mask의 종류를 다양화 하는 것을 목표로 함
   - 1단계에서 학습된 신뢰도 높은 Mask를 작업 화면에 표시
   - Annotator들은 그 외 Object를 작업
   - 새로 취득한 Data로 점진적 모델 학습(5회)
   - 180k 이미지로 부터 5.9M Mask 취득  
3. `Fully automatic`
   - 완전 자동화된 Annotation 단계
   - 이미지에 32 x 32 Regular Grid Point를 각 포인트 마다 대응되는 Mask가 할당된다.
   - 그 중에서 IoU가 높은 Mask만 남긴다.
   - 중복된 Mask 제거 등 후처리 작업 진행
   - 1.1M 이미지로 부터 1.1B Mask 취득

### 모델 구조

![image.png](https://github.com/facebookresearch/segment-anything/raw/main/assets/model_diagram.png?raw=true)
- Image encoder output인 image embedding, Prompt encoder output인 prompt embedding을 `두개의 embedding을 Lightweight Mask decoder에서 결합하여 Mask 예측`
- Prompt encoder + Mask decoder는 50ms 이내로 수행

- Powerful Image Encoder
- Prompt Encoder: Sparse/Dense Prompt로 나뉜다.
  - Sparse Prompt : Points, Boxes, Text를 임베딩 한다.
  - Dense Prompt : Mask을 임베딩 한다.
- Mask Decoder

※ 참고로 오픈소스에서는 Text를 임베딩하는 기능은 없다.

#### Image Encoder

#### Prompt Encoder

In [1]:
# segment-anything\segment_anything\modeling\prompt_encoder.py
class PromptEncoder(nn.Module):

    def __init__(
        self,
        embed_dim: int,
        image_embedding_size: Tuple[int, int],
        input_image_size: Tuple[int, int],
        mask_in_chans: int,
        activation: Type[nn.Module] = nn.GELU,
    ) -> None:
        super().__init__()
    
        self.mask_downscaling = nn.Sequential(
            nn.Conv2d(1, mask_in_chans // 4, kernel_size=2, stride=2),
            LayerNorm2d(mask_in_chans//4),
            activation(),
            nn.Conv2d(mask_in_chans // 4, mask_in_chans, kernel_size=2, stride=2),
            LayerNorm2d(mask_in_chans),
            activation()
            nn.Conv2d(mask_in_chans, embed_dim, kernel_size=1),
        )

    def _get_batch_size(
        self,
        points: Optional[Tuple[torch.Tensor, torch.Tensor]],
        boxes: Optional[torch.Tensor],
        masks: Optional[torch.Tensor],
    ) -> int:
        """Gets the batch size of the output given the batch size of the input
        prompts."""
        if points is not None:
            return points[0].shape[0]
        elif boxes is not None:
            return boxes.shape[0]
        elif masks is not None:
            return masks.shape[0]
        else:
            return 1



    def _embed_masks(self, masks: torch.Tensor) -> torch.Tensor:
        """Embeds mask inputs."""
        mask_embedding = self.mask_downscaling(masks)
        return mask_embedding

    def _get_dive(self) -> torch.device:
        return self.point_embeddings[0].weight.device

    def forward(
        self,
        points: Optional[Tuple[torch.Tensor, torch.Tensor]],
        boxes: Optional[torch.Tensor],
        masks: Optional[torch.Tensor],
    ) -> Tuple[torch.Tensor, torch.Tensor]:
    """Embeds different types of prompts, returning both sparse and dense
        embeddings.

        Arguments:
          points (tuple(torch.Tensor, torch.Tensor) or none): point coordinates
            and labels to embed.
          boxes (torch.Tensor or none): boxes to embed
          masks (torch.Tensor or none): masks to embed

        Returns:
          torch.Tensor: sparse embeddings for the points and boxes, with shape
            BxNx(embed_dim), where N is determined by the number of input points
            and boxes.
            : Points, Boxes의 sparse 임베딩
          torch.Tensor: dense embeddings for the masks, in the shape
            Bx(embed_dim)x(embed_H)x(embed_W)
            : Mask 임베딩
    """
    bs = self._get_batch_size(points, boxes, masks)
    sparse_embeddings = torch.empty((bs, 0, self.embed_dim), device=self._get_device())

    if points is not None:
        coords, labels = points
        point_embeddings = self._embed_points(coords, labels, pad=(boxes is None))
        sparse_embeddings = torch.cat([sparse_embeddings, point_embeddings], dim=1)
    if boxes is not None:
        box_embeddings = self._embed_boxes(boxes)
        sparse_embeddings = torch.cat([sparse_embeddings, box_embeddings], dim=1)
    
    if masks is not None:
        dense_embeddings = self._embed_masks(masks)
    else:
        dense_embeddings = self.no_mask_embed.weight.reshape(1, -1, 1, 1)
            .expand(bs, -1, self.image_embedding_size[0], self.image_embedding_size[1])
    return sparse_embeddings, dense_embeddings


IndentationError: expected an indented block (267457045.py, line 11)

#### Mask Decoder
- Image embedding과 prompt embedding을 받아 마스크를 예측하는 부분.
- Transformer decoder block에 Prompt Self-attention과 Cross-attention을 양방향으로 활용
- ![img.jpg](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbFpHTu%2Fbtr8NcL9qZm%2FcxrpfTQaoVn1ytXq4fAPik%2Fimg.png)

#### 성능
- 256개 A100 GPU로 3 ~ 5일 학습
- 이미지 인코더는 A100에서 0.15s, 632M 파라미터 수.
- 프롬프트 인코더와 마스크 디코더는 CPU에서 0.050s 에 추론 가능하다, 4M 파라미터 수.
- 

Everything 기능이란?
- <참고 강의>: https://www.youtube.com/watch?v=KQ3haqbIaSk&t=2413s
- Everything 기능이란? : 1024개 점(32 x 32)을 64개 씩 16번 나눠서 수행
  - 다시말해 특별한 기능이 아니고, SAM을 16 배치로 나눠서 수행
- Mask와 iou_prediction 추론 및 필터링
  - iou thres 낮은 것 제거
  - stability(?) 낮은 것 제거
  - nms 겹치는 것 제거
- 각 마스크 hole(구멍이 뚤려 있는 마스크는 채워주고)과 island(작게 남아 있는 마스크는 제거)
- 강의자 말에 의하면 아래 코드에 Everything 기능 구현이 있다고 한다.
  - segment-anything\segment_anything\automatic_mask_generator.py

인상적인 후속 주제들
- Inpaint Anything
- CLIP 과의 결합 (자연어를 해석) 
  : (개인 의견) 자연어와 결합해서 
- Segmentation with tracking
  : (개인 의견) 데이터를 수집할때 편할 것 같다.

### Predictor
- segment-anything\segment_anything\predictor.py
- Sam 모델을 사용하기 위한 껍데기라고 보면 된다.

In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

import numpy as np
import torch

from segment_anything.modeling import Sam

from typing import Optional, Tuple

from .utils.transforms import ResizeLongestSide

class SamPredictor:
    def __init__(
        self,
        sam_model: Sam,
    ) -> None:
        """
        Uses SAM to calculate the image embedding for an image, and then
        allow repeated, efficient mask prediction given prompts.

        SAM을 사용하여 이미지에 대한 이미지 임베딩을 계산한 다음, 주어진 프롬프르에서
        반복적이고 효율적인 마스크 예측 허용

        Arguments:
          sam_model (Sam): The model to use for mask prediction.
                           마스크 예측에 사용할 모델
        """
        super().__init__()
        self.model = sam_model
        self.transform = ResizeLongestSide(sam_model.image_encoder.img_size)
        self.reset_image()

    def set_image(
        self,
        image: np.ndarray,
        image_format: str = "RGB",
    ) -> None:
        """
        Calculates the image embeddings for the provided image, allowing
        masks to be predicted with the 'predict' method.

        Arguments:
          image (np.ndarray): The image for calculating masks. Expects an
            image in HWC uint8 format, with pixel values in [0, 255].
          image_format (str): The color format of the image, in ['RGB', 'BGR'].
        """
        assert image_format in [
            "RGB",
            "BGR",
        ], f"image_format must be in ['RGB', 'BGR'], is {image_format}."
        if image_format != self.model.image_format:
            image = image[..., ::-1]

        # Transform the image to the form expected by the model
        input_image = self.transform.apply_image(image) # ?
        input_image_torch = torch.as_tensor(input_image, device=self.device)
        input_image_torch = input_image_torch.permute(2, 0, 1).contiguous()[None, :, :, :]

        self.set_torch_image(input_image_torch, image.shape[:2])

    @torch.no_grad() # gpu 안쓴다??
    def set_torch_image(
        self,
        transformed_image: torch.Tensor,
        original_image_size: Tuple[int, ...],
    ) -> None:
        """
        Calculates the image embeddings for the provided image, allowing
        masks to be predicted with the 'predict' method. Expects the input
        image to be already transformed to the format expected by the model.

        제공된 이미지의 임베딩을 계산하여, 'predict' 메소드로 마스크를 예측할 수 있다.
        입력 이미지가 모델에서 예상하는 형식으로 이미 변환되어 있을 것으로 예상
        정의되는 변수
        - self.original_size
        - self.input_size
        - self.features
        - self.is_image_set

        Arguments:
          transformed_image (torch.Tensor): The input image, with shape
            1x3xHxW, which has been transformed with ResizeLongestSide.
          original_image_size (tuple(int, int)): The size of the image
            before transformation, in (H, W) format.

            ResizeLongestSide 클래스에 의해 변환된 1x3xHxW 모양 이미지
        """
        assert (
            len(transformed_image.shape) == 4 
            and transformed_image[1] == 3
            and max(*transformed_image.shape[2:]) == self.model.image_encoder.img_size
        ), f"set_torch_image input must be BCHW with long side {self.model.image_encoder.img_size}."
        self.reset_image()

        self.original_size = original_image_size
        self.input_size = tuple(transform_image.shape[-2:]) # HW
        input_image = self.model.preprocess(transformed_image)
        self.features = self.model.image_encoder(input_image) # 임베딩 된것
        self.is_image_set = True

def predict(
        self,
        point_coords: Optional[np.ndarray] = None,
        point_labels: Optional[np.ndarray] = None,
        box: Optional[np.ndarray] = None,
        mask_input: Optional[np.ndarray] = None,
        multimask_output: bool = True,
        return_logits: bool = False,
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Predict masks for the given input prompts, using the currently set image.

        주어진 입력 프롬프트에서, 마스크를 예측

        Arguments:
          point_coords (np.ndarray or None): A Nx2 array of point prompts to the
            model. Each point is in (X,Y) in pixels.

          point_labels (np.ndarray or None): A length N array of labels for the
            point prompts. 1 indicates a foreground point and 0 indicates a
            background point.

            N개 길이 1은 전경, 0은 배경을 가리킨다.

          box (np.ndarray or None): A length 4 array given a box prompt to the
            model, in XYXY format.

            XYXY 포맷으로 주어진 박스

          mask_input (np.ndarray): A low resolution mask input to the model, typically
            coming from a previous prediction iteration. Has form 1xHxW, where
            for SAM, H=W=256.

            이전 예측 반복에서 나오는 모델에 대한 저 해상도 마스크 입력. 1 x H x W
            SAM의 경우 H=W=256

          multimask_output (bool): If true, the model will return three masks.
            For ambiguous input prompts (such as a single click), this will often
            produce better masks than a single prediction. If only a single
            mask is needed, the model's predicted quality score can be used
            to select the best mask. For non-ambiguous prompts, such as multiple
            input prompts, multimask_output=False can give better results.

            true인 경우 3개 마스크를 반환
            모호한 입력 프롬프트의 경우(단일 클릭) 더 나은 마스트 생성(?)
            단일 마스크만 필요한 경우, 예측 퀄리티를 사용하여 최상의 마스크 선택.
            다중 입력 프롬프트 같이 모호하지 않은 경우는, multimask_output=False가 더 나은 결과를 제공할 수 있다.

          return_logits (bool): If true, returns un-thresholded masks logits
            instead of a binary mask.

            true인 경우 바이너리 마스크 대신 마스크 로짓을 반환

        Returns:
          (np.ndarray): The output masks in CxHxW format, where C is the
            number of masks, and (H, W) is the original image size.

          (np.ndarray): An array of length C containing the model's
            predictions for the quality of each mask.

          (np.ndarray): An array of shape CxHxW, where C is the number
            of masks and H=W=256. These low resolution logits can be passed to
            a subsequent iteration as mask input.

          masks: CxHxW 형식의 출력 마스크입니다. 여기서 C는 마스크 수이고 (H, W)는 원본 이미지 크기입니다.

          scores: 각 마스크 품질에 대한 모델의 예측을 포함하는 길이 C의 배열
          
          logits: CxHxW 모양의 배열. C는 마스크 수이고, H=W=256. 저해상도 logits는 마스크 입력으로 후속 반복 작업에 전달될 수 있다.
        """
        if not self.is_image_set:
            raise RuntimeError("An image must be set with .set_image(...) before mask prediction.")

        # Transform input prompts
        coords_torch, labels_torch, box_torch, mask_input_torch = None, None, None, None
        
        if point_coords is not None:
            assert (
                point_labels is not None
            ), "point_labels must be supplied if point_coords is supplied."

            point_coords = self.transform.apply_coords(point_coords, self.original_size) # 포인트를 (이미지를 늘리듯) 타켓 길이에 맞춰 비율 조정해 준다.
            coords_torch = torch.as_tensor(point_coords, dtype=torch.float, device=self.device)
            labels_torch = torch.as_tensor(point_labels, dtype=torch.int, device=self.device)
            coords_torch, labels_torch = coords_torch[None, :, :], labels_torch[None, :]
        
        if box is not None:
            box = self.transform.apply_boxes(box, self.original_size) # 포인트를 (이미지를 늘리듯) 타켓 길이에 맞춰 비율 조정해 준다.
            box_torch = torch.as_tensor(box, dtype=torch.float, device=self.device)
            box_torch = box_torch[None, :]
        
        # ★★★ 첫번째 예측할 때는 입력으로 안들어 오고, 
        # 모호한 입력에 대해 이전 마스크를 입력으로 줄 때 사용된다.
        if mask_input is not None: 
            mask_input_torch = torch.as_tensor(mask_input, dtype=torch.float, device=self.device)
            mask_input_torch = mask_input_torch[None, :, :, :]

        masks, iou_predictions, low_res_masks = self.predict_torch(
            coords_torch,
            labels_torch,
            box_torch,
            mask_input_torch,
            multimask_output,
            return_logits=return_logits,
        )

        masks = masks[0].detach().cpu().numpy()
        iou_predictions = iou_predictions[0].detach().cpu().numpy()
        low_res_masks = low_res_masks[0].detach().cpu().numpy()
        return masks, iou_predictions, low_res_masks


In [ ]:
def select_masks(
    masks: np.ndarray, iou_preds: np.ndarray, num_points: int
) -> Tuple [np.ndarray, np.ndarray]:
    # Determine if we should return the multiclick mask or not from the number of points.
    # The reweighting is used to avoid control flow.
    # Reference: https://github.com/facebookresearch/segment-anything/blob/6fdee8f2727f4506cfbbe553e23b895e27956588/segment_anything/utils/onnx.py#L92-L105
    score_reweight = np.array([1000] + [0] * 2)
    score = iou_preds + (num_points - 2.5) * score_reweight
    best_idx = np.argmax(score)
    masks = np.expand_dims(masks[best_idx, :, :], axis=-1)
    iou_preds = np.expand_dims(iou_preds[best_idx], axis=0)
    return masks, iou_preds


#### 사용하는 Transformer 블록

#### 사용하는 Attention

In [ ]:
class Attention(nn.Module):
    """Multi-head Attention block with relative position embeddings."""

    def __init__(
        self,
        dim: int,
        num_heads: int = 8,
        qkv_bias: bool = True,
        use_rel_pos: bool = False,
        rel_pos_zero_init: bool = True,
        input_size: Optional[Tuple[int, int]] = None,
    ) -> None:
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim**-0.5

        self.qkv = nn.Linear(dim, dim*3, bias=qkv_bias)
        self.proj = nn.Linear(dim, dim)   

        self.use_rel_pos = use_rel_pos
        if self.use_rel_pos:
            assert(
                input_size is not None
            ), "Input size must be provided if using relative positional encoding."
            # 상대 포지셔닝 임베딩을 초기화
            self.rel_pos_h = nn.Parameter(torch.zeros(2 * input_size[0] - 1, head_dim))
            self.rel_pos_w = nn.Parameter(torch.zeros(2 * input_size[1] - 1, head_dim))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, H, W, _ = x.shape
        # qkv with shape (3, B, nHead, H * W, C)
        qkv = self.qkv(x).reshape(B, H * W, 3, self.num_heads, -1)
        # q, k, v with shape (B * nHead, H * W, C)
        q, k, v = qkv.reshape(3, B * self.num_heads, H * W, -1).unbind(0)

        attn = (q * self.scale) @ K.transpose(2, -1)

        if self.use_rel_pos:
            attn = add_decomposed_rel_pos(attn, q, self.rel_pos_h, self.rel_pos_w, (H, W), (H, W))

        attn = attn.softmax(dim=-1)
        x = (attn @ v).view(B, self.num_heads, H, W, -1).permute(0, 2, 3, 1, 4)
        x = x.reshape(B, H, W, -1)
        x = self.proj(x)

        return x

In [ ]:
def get_rel_pos(q_size: int, k_size: int, rel_pos: torch.Tensor) -> torch.Tensor:
    """
    Get relative positional embeddings according to the relative positions of
        query and key sizes.
    
    q, k 사이즈의 상대 포지션에 따른 상대 포지셔닝 임베딩을 구한다. 
    
    Args:
        q_size (int): size of query q.
        k_size (int): size of key k.
        rel_pos (Tensor): relative position embeddings (L, C).

    Returns:
        Extracted positional embeddings according to relative positions.
    """
    max_rel_dist = int(2 * max(q_size, k_size) - 1) # 최대 상대 거리??
    # Interpolate rel pos if needed.
    if rel_pos.shape[0] != max_rel_dist:
        # Interpolate rel pos.
        rel_pos_resized = F.Interpolate(
            rel_pos.reshape(1, rel_pos.shape[0], -1).permute(0, 2, 1) # (1, -1, rel_pos.shape[0])
            size=max_rel_dist,
            mode="linear",
        )
        rel_pos_resized = rel_pos_resized.reshape(-1, max_rel_dist).permute(1, 0)
    else:
        rel_pos_resized = rel_pos
        
    # Scale the coords with short length if shapes for q and k are different.
    # Query(Q)의 각 위치에 대한 상대적인 좌표를 계산합니다. q_size보다 k_size가 클 경우, 각 Query 위치를 더 큰 간격으로 매핑
    q_coords = torch.arange(q_size)[:, None] * max(k_size / q_size, 1.0) # 열 벡터로 변경
    k_coords = torch.arange(k_size)[None, :] * max(q_size / k_size, 1.0) # 핼 벡터로 변경
    relative_coords = (q_coords - k_coords) + (k_size - 1) * max(q_size / k_size, 1.0)

    # Query와 Key에 대한 상대적인 좌표를 계산하고, 상대적인 위치 정보를 담은 relative_coords라는 텐서를 생성하는 것
    return rel_pos_resized[relative_coords.long()]

In [ ]:
def add_decomposed_rel_pos(
    attn: torch.Tensor,
    q: torch.Tensor,
    rel_pos_h: torch.Tensor,
    rel_pos_w: torch.Tensor,
    q_size: Tuple[int, int],
    k_size: Tuple[int, int],
) -> torch.Tensor:
    """
    Calculate decomposed Relative Positional Embeddings from :paper:`mvitv2`.
    https://github.com/facebookresearch/mvit/blob/19786631e330df9f3622e5402b4a419a263a2c80/mvit/models/attention.py   # noqa B950
    Args:
        attn (Tensor): attention map.
        q (Tensor): query q in the attention layer with shape (B, q_h * q_w, C).
        rel_pos_h (Tensor): relative position embeddings (Lh, C) for height axis.
        rel_pos_w (Tensor): relative position embeddings (Lw, C) for width axis.
        q_size (Tuple): spatial sequence size of query q with (q_h, q_w).
        k_size (Tuple): spatial sequence size of key k with (k_h, k_w).

    Returns:
        attn (Tensor): attention map with added relative positional embeddings.
    """
    q_h, q_w = q_size
    k_h, k_w = k_size
    Rh = get_rel_pos(q_h, k_h, rel_pos_h)
    Rw = get_rel_pos(q_w, k_w, rel_pos_w)

    B, _, dim = q.shape
    r_q = q.reshape(B, q_h, q_w, dim) # (B, H, W, C)
    rel_h = torch.einsum("bhwc,hkc->bhwk", r_q, Rh)
    rel_w = torch.einsum("bhwc,wkc->bhwk", r_q, Rw)

    # rel_h[:, :, :, :, None]과 rel_w[:, :, :, None, :]는 각각 Query와 Key에 대한 상대적인 위치를 나타내는 텐서들
    # 이 텐서들은 브로드캐스팅을 활용하여 attn 텐서의 각 위치에 대해 상대적인 위치 정보를 더합니다
    # 
    attn = (
        attn.view(B, q_h, q_w, k_h, k_w) + rel_h[:, :, :, :, None] + rel_w[:, :, :, None, :]
    ).view(B, q_h * q_w, k_h * k_w)

    # (B, q_h * q_w, k_h * k_w) 모양으로 변경합니다. 이는 어텐션 스코어 행렬을 표현하는 것으로, 각 Query와 Key의 조합에 대한 유사도를 나타낸다
    # 이 코드는 주어진 어텐션 스코어 행렬 attn에 상대적인 위치 정보를 더하여 최종 어텐션 스코어 행렬을 재구성하는 작업을 수행
    return attn   